# 2025 Championship Analysis

In this notebook I will take a look at the 2025 WDC championship mainly.

The idea is: McLaren easily won WCC but lost (or almost lost) the WDC. I'm starting the work on this before the championship actually finished.

In [ ]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
import os
from manim import *

In [ ]:
# Caminho absoluto da raiz do projeto
project_root = Path.cwd().parents[1]  # se o notebook está em /notebooks
print("Diretório atual:", Path.cwd())
os.chdir(project_root)

from src.modules.data_processing.db_reader import DbReader
from src.analysis.championship.plotter import *
from src.modules.animation.manim_constructors import LineChampionshipChart
from src.analysis.data_viz.plotter import *
f1_db = DbReader()

In [ ]:
# Drivers Standings

df_wdc = f1_db.run_query_file('data/db_queries/drivers_standings.sql')
df_wdc_2025 = df_wdc[df_wdc['year']==2025]

In [ ]:
df_wdc

In [ ]:
df_wcc = f1_db.run_query_file('data/db_queries/constructors_standings.sql')
df_wcc_2025 = df_wcc[df_wcc['year']==2025].copy()

In [ ]:
df_wcc

In [ ]:
df_races = f1_db.run_query_file('data/db_queries/race_results_report.sql')
df_races = df_races[df_races['year']==2025]

In [ ]:
df_races

## Championship Evolution: 

### WCC

McLaren easily won the WCC. Let's take a look:

In [ ]:
plot_wcc(
    df_campeonato=df_wcc_2025,
    )

In [ ]:
plot_wcc(
    df_campeonato=df_wcc_2025,
    times_destaque=['McLaren', 'Red Bull', 'Ferrari', 'Mercedes']
    )

Above we can see the evolution of the 2025 WCC.

By that we can see that McLaren clearly had the advantage along the year.

The only other teams fighting with eachother for the 3 top positions are Mercedes, Red Bull and Ferrari (somehow).

### Manim

Animated WCC chart:

#### Adjustments for animated chart:

In [ ]:
# Correção dos nomes dos construtores para coincidir com o dicionário de cores

dict_fix_constructor_names = {
    'RB F1 Team': 'VCARB',
    'Haas F1 Team': 'Haas',
    'Alpine F1 Team': 'Alpine'
}

df_wcc_2025['constructor_name'] = df_wcc_2025['constructor_name'].replace(dict_fix_constructor_names)

In [ ]:
dict_team_colors = {
    "Red Bull": "#3671C6",       # Azul Clássico
    "Mercedes": "#27F4D2",       # Verde/Ciano Petronas (brilha bem no escuro)
    "Ferrari": "#E80020",        # Rosso Corsa
    "McLaren": "#FF8000",        # Laranja Papaya
    "Aston Martin": "#229971",   # British Racing Green
    "Alpine": "#0090FF",         # Azul Alpine (Nota: às vezes usam rosa da BWT)
    "Williams": "#64C4FF",       # Azul Celeste
    "VCARB": "#6692FF",          # Azul 'Visa Cash App' (mais claro que a RBR)
    "Sauber": "#52E252",         # Verde Neon (Kick/Stake branding)
    "Haas": "#B6BABD",           # Cinza/Branco (neutro)
    "Cooper-Climax": '#004225'   # Verde Escuro (clássico)
}

In [ ]:
race_list = [race[0] for race in df_wcc_2025[['round_id', 'race_name']].drop_duplicates().sort_values('round_id')[['race_name']].values]

race_list = [race.replace('Grand Prix', 'GP') for race in race_list]

In [ ]:
y_max = df_wcc_2025['points'].max() + 10
x_max = df_wcc_2025['round_id'].max() - df_wcc_2025['round_id'].min() + 1
df_wcc_2025_grouped = df_wcc_2025.groupby('constructor_name')

In [ ]:
dict_axis_config = {
    "include_numbers": True, 
    "color": GRAY, 
    'include_ticks': False, 
    'include_tip': False,
    'font_size': 20,
}

In [ ]:
team_file_map = {
    "Red Bull": "red_bull.png",
    "Ferrari": "ferrari.png",
    "McLaren": "mclaren.png",
    "Mercedes": "mercedes.png",
    "Aston Martin": "aston_martin.png",
    "Alpine": "alpine.png",
    "Williams": "williams.png",
    "VCARB": "vcarb.png",
    "Sauber": "sauber.png",
    "Haas": "haas.png"
}

%%manim -pqh -t -v WARNING WCC_Animated_Chart


class WCC_Animated_Chart(LineChampionshipChart):
    def __init__(self, **kwargs):
        # Preparando o caminho dos logos como string
        logos_path_str = str(project_root / 'notebooks/2025_championship/assets/team_logos/')
        
        super().__init__(
            # --- SEUS DADOS ---
            df_grouped=df_wcc_2025_grouped,
            race_list=race_list,
            team_colors=dict_team_colors,
            x_max=x_max,
            y_max=y_max,
            
            # --- CUSTOMIZAÇÃO ---
            # Aqui passamos o dicionário de eixos que você pediu
            axis_config=dict_axis_config, 
            
            # Logos
            logos_dir=logos_path_str,
            logo_map=team_file_map,
            
            # --- CONTROLES DE VÍDEO ---
            static_mode=False,        # False = Gera o vídeo animado
            show_gap=True,            # Mostra a linha de gap
            transparent_bg=True,      # Fundo transparente (Requer flag -t acima)
            
            # Estilo
            color_highlight=dict_team_colors.get('McLaren', "#FF8000"),
            
            **kwargs
        )

### Let's take a look at points percentage:

In [ ]:
df_wcc_2025_final_round = df_wcc_2025[df_wcc_2025['round_id'] == df_wcc_2025['round_id'].max()].copy()

df_wcc_2025_final_round['points_percentage'] = df_wcc_2025_final_round['points'] / df_wcc_2025_final_round['points'].sum() * 100

df_wcc_2025_final_round

In [ ]:
graf_barras_padrao(
    df_dados=df_wcc_2025_final_round,
    x_col='constructor_name',
    y_col='points_percentage',
    hue_col=None,
    cores_map=dict_team_colors,
    titulo="Points Percentage by Constructor in Final Round 2025",
    xlabel="Constructor",
    ylabel="Points Percentage (%)",
    fmt_rotulo='%.2f%%'
)

Above we can see that McLaren scored way more points than any other team in terms of percentage, and also how tight the battle was for second place between Mercedes, Red Bull and Ferrari.

And also, I feel kinda sorry for Alpine.

Let's look how this McLaren "domination" looks like if we compare it to historical values:

In [ ]:
# Primeiro vou ranquear os maiores percentuais históricos de pontos no WCC

df_wcc['last_round_id_in_year'] = df_wcc.groupby('year')['round_id'].transform('max')

df_wcc_final_round = df_wcc[df_wcc['round_id'] == df_wcc['last_round_id_in_year']].copy()

df_wcc_final_round['total_points_score_in_year'] = df_wcc_final_round.groupby('year')['points'].transform('sum')

df_wcc_final_round['points_percentage'] = df_wcc_final_round['points'] / df_wcc_final_round['total_points_score_in_year'] * 100

df_wcc_final_round['highest_percentage_in_year'] = df_wcc_final_round.groupby('year')['points_percentage'].transform('max')

df_wcc_final_round_highest_pctgs = df_wcc_final_round[df_wcc_final_round['points_percentage'] == df_wcc_final_round['highest_percentage_in_year']].copy()

df_wcc_final_round_highest_pctgs['constructor_name_year'] = df_wcc_final_round_highest_pctgs['constructor_name'] + ' (' + df_wcc_final_round_highest_pctgs['year'].astype(str) + ')'

df_wcc_final_round_highest_pctgs = df_wcc_final_round_highest_pctgs.sort_values(['highest_percentage_in_year'], ascending=False)

df_wcc_final_round_highest_pctgs

In [ ]:
df_wcc_final_round_highest_pctgs['constructor_name_year'].head(10).unique()

In [ ]:
dict_map_cores_highest_pctgs = {}

for name_year in df_wcc_final_round_highest_pctgs['constructor_name_year'].unique():
    for equipe, cor in dict_team_colors.items():
        if equipe in name_year:
            dict_map_cores_highest_pctgs[name_year] = cor

graf_barras_padrao(
    df_dados=df_wcc_final_round_highest_pctgs.head(10),
    x_col='constructor_name_year',
    y_col='points_percentage',
    hue_col=None,
    cores_map=dict_map_cores_highest_pctgs,
    titulo="Top 10 Highest Points Percentage by Constructor in a Season (WCC History)",
    xlabel="Constructor (Year)",
    ylabel="Points Percentage (%)",
    fmt_rotulo='%.2f%%'
)

Historically speaking, there have been mopre dominant performances, but all from completely different eras of F1.

If we take a look at only the past 20 years:

In [ ]:
graf_barras_padrao(
    df_dados=df_wcc_final_round_highest_pctgs[df_wcc_final_round_highest_pctgs['year']>2004].head(10),
    x_col='constructor_name_year',
    y_col='points_percentage',
    hue_col=None,
    cores_map=dict_map_cores_highest_pctgs,
    titulo="Top 10 Highest Points Percentage by Constructor in a Season (WCC History)",
    xlabel="Constructor (Year)",
    ylabel="Points Percentage (%)",
    fmt_rotulo='%.2f%%'
)

Still not in the top 10, but not that far off. But anyways, despite not being one of the most dominant performances historically speaking (*coff* could have been considering some stuff that happened in the season *coff*) it was still a breeze.

What was interesting was the WDC. Let's take a look at how that went:

### WDC

In contrast, if we take a look at the WDC

In [ ]:
plot_wdc(df_campeonato=df_wdc_2025)

In [ ]:
plot_wdc(
    df_campeonato=df_wdc_2025,
    pilotos_destaque=[
        'Max Verstappen',
        'Lando Norris',
        'Oscar Piastri'
    ],
    save_fig=True,
    save_path='grafs/2025_championship'
    )